# Basic Configs

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Imports

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import datasets, transforms
from torch.utils import data

import numpy as np
import matplotlib.pyplot as plt

## Global Variables

In [3]:
cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")
cpu = torch.device("cpu")
batch_size = 128
num_workers = 4

## Load Repo
- Rename the folder to `project`

In [ ]:
# !rm -rf project

In [4]:
!git clone https://github.com/effie-0/IDL-Project.git

Cloning into 'IDL-Project'...
remote: Enumerating objects: 253, done.
remote: Counting objects: 100% (253/253), done.
remote: Compressing objects: 100% (187/187), done.
remote: Total 253 (delta 132), reused 132 (delta 54), pack-reused 0
Receiving objects: 100% (253/253), 1.60 MiB | 3.33 MiB/s, done.
Resolving deltas: 100% (132/132), done.


In [5]:
!mv IDL-Project project

In [32]:
!cd project; git checkout attack

Branch 'attack' set up to track remote branch 'attack' from 'origin'.
Switched to a new branch 'attack'


# Load Functions

In [6]:
from project.summarize.load_data import AdvDataset, get_data

## Load Dataset

In [7]:
trainset, trainloader, testset, testloader = get_data(batch_size)

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


## Load Models

In [8]:
from project.summarize.classifier import ResidualBlock, ResNet18

In [9]:
resnet18 = ResNet18(ResidualBlock)
another_model = ResNet18(ResidualBlock)

### Load Trained Models

In [10]:
criterion = nn.CrossEntropyLoss()

In [13]:
from project.summarize.normal_train import evaluate

In [21]:
resnet18_path = "/content/gdrive/MyDrive/18-786 IDL/Project/models/model_0.pth"
resnet18.load_state_dict(torch.load(resnet18_path)['model_state_dict'])

<All keys matched successfully>

In [22]:
resnet18.to(device)
_, acc = evaluate(resnet18, testloader, criterion, device)
print('acc = ', acc)
resnet18.to(cpu)

acc =  85.34


ResNet18(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (layer1): Sequential(
    (0): ResidualBlock(
      (left): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (shortcut): Sequential()
    )
    (1): ResidualBlock(
      (left): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(i

In [23]:
another_model_path = "/content/gdrive/MyDrive/18-786 IDL/Project/models/model_15.pth"
another_model.load_state_dict(torch.load(another_model_path)['model_state_dict'])

<All keys matched successfully>

In [24]:
another_model.to(device)
_, acc = evaluate(another_model, testloader, criterion, device)
print('acc = ', acc)
another_model.to(cpu)

acc =  85.1


ResNet18(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (layer1): Sequential(
    (0): ResidualBlock(
      (left): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (shortcut): Sequential()
    )
    (1): ResidualBlock(
      (left): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(i

# Generate the Adversarial Samples

In [25]:
!pip install adversarial-robustness-toolbox

     |████████████████████████████████| 890kB 13.5MB/s 
     |████████████████████████████████| 235kB 50.8MB/s 
     |████████████████████████████████| 21.0MB 1.4MB/s 
     |████████████████████████████████| 747kB 69.0MB/s 


In [26]:
from art.attacks.evasion import FastGradientMethod, ProjectedGradientDescentPyTorch, CarliniLInfMethod, DeepFool
from art.estimators.classification import PyTorchClassifier

## Prepare Attacks

## Eps = 0.03

In [27]:
def generate_attack(test_loader, classifier, attack):
    adv_examples = []
    # Loop over all examples in test set
    for image, target in test_loader:
        # Send the data and label to the device
        # image, target = image.to(device), target.to(device)

        # Call Attack
        adv_data = attack.generate(x=image, y=target)

        # adv_data = adv_data.squeeze().detach().cpu().numpy()
        # adv_examples.append( (target.flatten().detach().cpu().numpy(), adv_data) )
        adv_examples.append( (target.flatten().numpy(), adv_data) )
        # pred_list.append((init_pred.flatten().detach().cpu().numpy(), final_pred.flatten().detach().cpu().numpy()))
    
    label = [j for i in adv_examples for j in i[0]]
    adv_ex = [j for i in adv_examples for j in i[1]]

    dataset = AdvDataset(adv_ex, label)
    loader = data.DataLoader(dataset,
                             batch_size=batch_size, 
                             shuffle=False,
                             num_workers=num_workers)

    # Return the accuracy and an adversarial example
    return dataset, loader

In [29]:
shape = trainset[0][0].shape
eps = 0.03

model = resnet18
model.to(device)
model.eval()
classifier = PyTorchClassifier(
    model=model,
    loss=criterion,
    input_shape=shape,
    nb_classes=10)
attack_samples = {}

print('FGM')
attack = FastGradientMethod(estimator=classifier, eps=eps)
dataset, loader = generate_attack(testloader, classifier, attack)
attack_samples['FGM'] = dataset, loader

_, acc = evaluate(model, loader, criterion, device)
print('acc = ', acc)

print('PGD')
attack = ProjectedGradientDescentPyTorch(estimator=classifier, eps=eps, eps_step=0.01, max_iter=10)
dataset, loader = generate_attack(testloader, classifier, attack)
attack_samples['PGD'] = dataset, loader

_, acc = evaluate(model, loader, criterion, device)
print('acc = ', acc)

print('C&W')
attack = CarliniLInfMethod(classifier=classifier, eps=eps, max_iter=10)
dataset, loader = generate_attack(testloader, classifier, attack)
attack_samples['C&W'] = dataset, loader

_, acc = evaluate(model, loader, criterion, device)
print('acc = ', acc)

model.to(cpu)
del classifier
torch.cuda.empty_cache()

FGM
acc =  37.16
PGD


PGD - Batches:   0%|          | 0/4 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/art/estimators/classification/pytorch.py:525: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_grad = torch.tensor(x).to(self._device)
/usr/local/lib/python3.6/dist-packages/art/estimators/classification/pytorch.py:526: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_grad = torch.tensor(y).to(self._device)


acc =  26.619999999999997
C&W


C&W L_inf: 100%|██████████| 1/1 [00:00<00:00,  1.14it/s]


acc =  29.17


In [33]:
from project.summarize.generate_attack import generate_mi_fgsm_attack, generate_fgsm_attack

In [34]:
model = resnet18
model.to(device)
model.eval()

print('FGSM')
acc, adv_ex, adv_label = generate_fgsm_attack(model, criterion, testloader, testset, eps, device)
dataset = AdvDataset(adv_ex, adv_label)
loader = data.DataLoader(dataset, batch_size=batch_size, 
                         shuffle=False,
                         num_workers=num_workers)
attack_samples['FGSM'] = dataset, loader

print('MI-FGSM')
acc, adv_ex, adv_label = generate_mi_fgsm_attack(model, criterion, testloader, testset, eps, device)
dataset = AdvDataset(adv_ex, adv_label)
loader = data.DataLoader(dataset, batch_size=batch_size, 
                         shuffle=False,
                         num_workers=num_workers)
attack_samples['MI-FGSM'] = dataset, loader

model.to(cpu)
torch.cuda.empty_cache()

FGSM
Epsilon: 0.03	Test Accuracy = 3716 / 10000 = 0.3716
MI-FGSM
Epsilon: 0.03	Test Accuracy = 3103 / 10000 = 0.3103


# 4x4 Matrix

In [35]:
another_model.to(device)
for attack_name, (dataset, loader) in attack_samples.items():
    loss, acc = evaluate(another_model, loader, criterion, device)
    print('attack: ', attack_name, ';  acc = ', acc)
another_model.to(cpu)
torch.cuda.empty_cache()

attack:  FGM ;  acc =  73.92999999999999
attack:  PGD ;  acc =  73.72999999999999
attack:  C&W ;  acc =  81.97
attack:  FGSM ;  acc =  76.7
attack:  MI-FGSM ;  acc =  81.77
